In [0]:
# Importing the modules that we'll need for this exercise
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
from requests_html import AsyncHTMLSession
import asyncio

In [0]:
# %pip install selenium ### I used this code to install selenium on my cluster. 

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager

In [0]:
# The hard coded list of URLs
urls = [
    "https://www.usnews.com/best-colleges/washington-university-in-st-louis-2520",
    "https://www.usnews.com/best-colleges/saint-louis-university-2506",
    "https://www.usnews.com/best-colleges/umsl-2519",
    "https://www.usnews.com/best-colleges/webster-university-2521"
]

# Adding headers to help with scraping. (I used AI, specifically Claude, to help me with this as USNews was giving me some scraping errors)
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
    'Accept-Language': 'en-US,en;q=0.5',
    'Accept-Encoding': 'gzip, deflate, br',
    'Connection': 'keep-alive',
    'Upgrade-Insecure-Requests': '1'
}


# Store all the scraped data
scraped_data = []

session = HTMLSession()

## WHAT FOLLOWS IS DEFUCT CODE ONLY FOR REFERENCE
###### This next block is a defunct codeset. Upon my work, I learned the US News loads their pages dynamically, making scraping using requests unusable. I left this code in here for visability into my overall process and what prompted me to work through another avenue. 


In [0]:
%skip
### This is a defunct codeset. Upon my work, I learned the US News loads their pages dynamically, making scraping using requests unusable. I continue my 


# Scraping and parsing the data into the scraped_data list. 
for i, url in enumerate(urls):
    try: # scraping code
        print(f"Scraping {i+1}/{len(urls)}: {url}")

        response = requests.get(url, headers=headers, timeout=60)
        response.raise_for_status()

        soup = BeautifulSoup(response.text, 'html.parser')

        # Find the specific population element within the noted class. 
        element = soup.find('p', class_='MuiTypography-root MuiTypography-body1 css-1udwrrf')

        if element: 
            number = element.text.strip() # Gets the population number reported on the site. 
            number_clean = int(number.replace(',','')) # Removing the comma and turning the scraped element text into a number. 
        else: 
            number = None
            print(f" !! Element not found on {URL}")

        scraped_data.append({
            'url': url, 
            'population': number_clean
        })

        print(f" Extracted: {number} from {url}")

        # adding a delay between requests
        time.sleep(3)

    except Exception as e: # exception handling
        print(f" x Error with {url}: {e}")
        

In [0]:
%skip
# Configure Chrome options
chrome_options = Options()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_argument('--disable-gpu')

# ChromeDriverManager automatically downloads the right driver
driver = webdriver.Chrome(
    service=Service(ChromeDriverManager().install()),
    options=chrome_options
)

try:
    for i, url in enumerate(urls):
        try: 
            print(f"Scraping {i+1}/{len(urls)}: {url}")

            driver.get(url)

            # wait for the element to load. 
            element = WebDriverWait(driver, 20).until(
                EC.presence_of_element_located((
                    By.CSS_SELECTOR,
                    "p.MuiTypography-root.MuiTypography-body1.css-1udwrrf"
                ))
            )

            number = element.text.strip()
            number_clean = int(number.replace(',',''))
            
            print(f" Extracted: {number} from {url}")

            scraped_data.append({'url': url, 'students': number_clean})

            time.sleep(2)
        except Exception as e:
            print(f"Error with {url}: {e}")
finally:
    driver.quit()

In [0]:
%skip
for i, url in enumerate(urls):
    try:
        print(f"Scraping {i+1}/{len(urls)}: {url}")
        
        response = session.get(url)
        
        # Render JavaScript (this may take a moment)
        response.html.render(timeout=20)
        
        # Find the element
        elements = response.html.find('p.MuiTypography-root.MuiTypography-body1.css-1udwrrf')
        
        if elements:
            number = elements[0].text.strip()
            print(f"  ✓ Extracted: {number}")
        else:
            number = None
            print(f"  ⚠ Element not found")
            
        scraped_data.append({'url': url, 'value': number})
        
        time.sleep(2)
        
    except Exception as e:
        print(f"  ✗ Error: {e}")

In [0]:
async def scrape_url(session, url, index, total):
    try:
        print(f"Scraping {index}/{total}: {url}")
        
        response = await session.get(url)
        
        # Render JavaScript
        await response.html.arender(timeout=20)
        
        # Find the element
        elements = response.html.find('p.MuiTypography-root.MuiTypography-body1.css-1udwrrf')
        
        if elements:
            number = elements[0].text.strip()
            print(f"  ✓ Extracted: {number}")
        else:
            number = None
            print(f"  ⚠ Element not found")
            
        return {'url': url, 'value': number}
        
    except Exception as e:
        print(f"  ✗ Error: {e}")
        return {'url': url, 'value': None}

In [0]:
async def scrape_all():
    all_data = []
    asession = AsyncHTMLSession()
    
    for i, url in enumerate(urls, 1):
        result = await scrape_url(asession, url, i, len(urls))
        all_data.append(result)
        await asyncio.sleep(2)  # Delay between requests
    
    await asession.close()
    return all_data

In [0]:
# Use the existing event loop in Databricks
import nest_asyncio
nest_asyncio.apply()

all_data = asyncio.get_event_loop().run_until_complete(scrape_all())